In [1]:
import os
import matplotlib.pyplot as plt
import mne

In [3]:

path = 'Ar-EEG Dataset'

os.listdir(path)


folders = [f"{path}/{x}" for x in os.listdir(path) if os.path.isdir(os.path.join(path,x))]
folders = folders[2:]

folders


individuals = [] 

for folder in folders: 
    for individual in os.listdir(folder):
        individuals.append(f"{folder}/{individual}/eeg")


sesions = []
for individual in individuals:
    
    sesions.append(individual)

files = []
for sesion in sesions: 
    tempList = []
    for file in os.listdir(sesion):
        tempList.append(f"{sesion}/{file}")
    files.append(tempList)
    


In [4]:
import numpy as np
signals = []
labels_arr =[] 
for file in files:
    eeg_file = file[0]
    vhdr_file = file[1]
    vmrk_file = files[-1]
    
    raw =mne.io.read_raw_brainvision(vhdr_file,preload=True)
    data = raw.get_data()  # EEG data (n_channels, n_timepoints)
    sfreq = raw.info['sfreq']  # Sampling frequency
    ch_names = raw.ch_names  # Channel names

    # Extract events
    events, event_id = mne.events_from_annotations(raw)

    # Get event times and codes
    event_times = events[:, 0]  # Sample indices where events occur
    event_codes = events[:, 2]  # Numerical event IDs

    # Reverse event_id dictionary to map event codes to labels
    event_label_map = {v: k for k, v in event_id.items()}  # {10001: 'Comment/Down', ...}

    # Create a labels array filled with "None" initially
    labels = np.array(["None"] * data.shape[1], dtype=object)  # One label per timepoint

    # Assign labels at event times
    for time, code in zip(event_times, event_codes):
        labels[time] = event_label_map.get(code, "Unknown")  # Assign label, default to "Unknown" if not found




    count = 0 
    for i in range(len(labels)):
        if (labels[i] != 'None' and i>=1000):
            signal_window = data[:, i-1000:i]  # Extract signal from all channels in the time window

            signals.append(signal_window)  # Add the signal window
            labels_arr.append(labels[i])  # Add the corresponding label

        

Extracting parameters from Ar-EEG Dataset/sub-1/ses-0/eeg/sub-1_ses-0_task-innerspeech_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 144553  =      0.000 ...   578.212 secs...
Used Annotations descriptions: [np.str_('Comment/Down'), np.str_('Comment/Left'), np.str_('Comment/Rest'), np.str_('Comment/Right'), np.str_('Comment/Select'), np.str_('Comment/Up'), np.str_('Comment/Wait'), np.str_('Comment/WarmUp')]
Extracting parameters from Ar-EEG Dataset/sub-1/ses-1/eeg/sub-1_ses-1_task-innerspeech_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 132835  =      0.000 ...   531.340 secs...
Used Annotations descriptions: [np.str_('Comment/Down'), np.str_('Comment/Left'), np.str_('Comment/Rest'), np.str_('Comment/Right'), np.str_('Comment/Select'), np.str_('Comment/Up'), np.str_('Comment/Wait'), np.str_('Comment/WarmUp')]
Extracting parameters from Ar-EEG Dataset/sub-1/ses-10/eeg/sub-1_ses-10_task-innerspeech_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 1

In [5]:
X = np.stack(signals)
y = np.stack(labels_arr)

In [6]:
print(X.shape)
print(y.shape)

(16632, 8, 1000)
(16632,)


In [7]:
np.unique(y)

array(['Comment/Down', 'Comment/Left', 'Comment/Rest', 'Comment/Right',
       'Comment/Select', 'Comment/Up', 'Comment/Wait'], dtype='<U14')

In [ ]:
y

In [8]:
np.save("../Data/raw_signals.npy",X)
np.save("../Data/labels.npy", y)